In [52]:
from test_single_sentence import SimpleExplainerEvaluator

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
evaluator = ExplainerEvaluator(model_name="cardiffnlp/twitter-xlm-roberta-base-sentiment", num_labels=3)

In [7]:
hatexdata, num_labels = evaluator.load_dataset("HateXplain")
print(f"hatexdata {hatexdata} num labels {num_labels}")

hatexdata <dataset_loaders.hatexplain.HateXplain object at 0x000001A4EF9171C0> num labels 3


In [8]:
hatexdata[178]["text"], hatexdata[178]["rationale"]

rationale label [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [], []]
final rationale [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
rationale label [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [], []]
final rationale [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


('if you ’ re an army that is republican you are not welcome here we hate you',
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [4]:
hatexdata._get_rationale(2)

ahdido
rationale label [[], [], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
final rationale [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]

In [11]:
moviereviews, num_labels = evaluator.load_dataset("MovieReviews")
print(f"moviereviews {moviereviews} num labels {num_labels}")

moviereviews <dataset_loaders.movie_reviews.MovieReviews object at 0x0000019C80535D30> num labels 2


In [61]:
# evaluator = SimpleExplainerEvaluator(model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=3)
evaluator = SimpleExplainerEvaluator(model_name="Hate-speech-CNERG/bert-base-uncased-hatexplain", num_labels=3)
# sentence = "You are the sweatest person, I wish I had known you before."
sentence= "<user> <user> why are you repeating yourself are you a little retarded"
# sentence = "<user> lol not me i don \u2019 t deal with bms not outside children \u2019 ss i leave that ghetto mess for everybody else \ud83d\ude02 \ud83d\ude02 \ud83d\ude02"
single_scores= evaluator.compute_score_single_sentence(sentence)
print(single_scores)

Hate-speech-CNERG/bert-base-uncased-hatexplain
{'hate speech': 0.027522409334778786, 'normal': 0.303344190120697, 'offensive': 0.6691333651542664}


In [63]:
explanations = evaluator.explain(sentence, target="offensive", normalize_scores=True)
evaluator.show_table(explanations)

Explainer:   0%|          | 0/3 [00:00<?, ?it/s]WARNING:root:Using hardcoded '[SEP]' as separator token.
d:\TUM courses\thesis- nlp\Code\myenv\lib\site-packages\captum\attr\_core\deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
Explainer:  67%|██████▋   | 2/3 [00:08<00:05,  5.03s/it]WARNING:root:Using hardcoded '[SEP]' as separator token.
d:\TUM courses\thesis- nlp\Code\myenv\lib\site-packages\captum\attr\_core\guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


,<_0,user_0,>_0,<_1,user_1,>_1,why,are_0,you_0,repeating,yourself,are_1,you_1,a,little,re,##tar,##ded
DeepLift,-0.06,0.04,0.01,0.08,0.04,0.03,0.04,0.03,-0.04,0.02,0.02,-0.05,0.00,0.01,0.01,-0.23,0.05,0.06
Integrated Gradient,-0.04,-0.08,-0.01,0.02,0.00,0.02,-0.02,-0.03,-0.05,-0.01,0.07,0.00,-0.03,-0.06,-0.06,0.23,-0.17,-0.03
Guided Backpropagation,0.05,0.01,-0.03,-0.04,-0.01,-0.00,0.01,-0.01,0.02,0.02,-0.02,0.02,0.02,0.04,-0.02,-0.18,-0.39,0.03


In [2]:
from explainers.integratedgradient import IntegratedGradientsExplainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from explainers.saliencymanager import SaliencyScoreManager
# model_name="cardiffnlp/twitter-xlm-roberta-base-sentiment"
model_name="Hate-speech-CNERG/bert-base-uncased-hatexplain"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cpu')

d:\TUM courses\thesis- nlp\Code\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# explainer = IntegratedGradientsExplainer(model, tokenizer)
# Create the Saliency Manager for generating explanations
manager = SaliencyScoreManager(
            model=model,
            tokenizer=tokenizer,
            explainer_class=IntegratedGradientsExplainer,
            device="cpu"
)
       

In [21]:
evaluator = ExplainerEvaluator(model_name=model_name, num_labels=3)

In [22]:
hatexplain, num_labels = evaluator.load_dataset("HateXplain")
moviereviews, num_labels = evaluator.load_dataset("MovieReviews")

In [4]:
from dataset_loaders import MovieReviews, HateXplain
import datasets
import os
import itertools
def load_dataset(dataset_name: str, **kwargs):
        """
        Load a dataset given its name.

        Args:
            dataset_name (str): Name of the dataset to load (e.g., "HateXplain").
            **kwargs: Additional arguments to pass to dataset loaders.

        Returns:
            dataset: Loaded dataset instance.
        """
        if dataset_name == "HateXplain":
            data = HateXplain(tokenizer, **kwargs)
            num_labels = 3  # HateXplain is a binary classification dataset
        elif dataset_name == "MovieReviews":
            data = MovieReviews(tokenizer, **kwargs)
            num_labels = 2  # MovieReviews is typically a binary classification dataset
        else:
            try:
                data = datasets.load_dataset(dataset_name)
                # Assuming we can infer num_labels from the dataset
                num_labels = len(set(example['label'] for example in data['train']))
            except:
                raise ValueError(f"Dataset {dataset_name} is not supported.")

        return data, num_labels

In [5]:
explainer_name = getattr(IntegratedGradientsExplainer, "NAME", IntegratedGradientsExplainer.__class__.__name__)
print(explainer_name)
model_name_file= 'bert_finetuned'
datasets_name= 'HateXplain'
# datasets_name=['HateXplain','MovieReviews']

Integrated Gradient


In [6]:
file_name = f"saliency_scores/{model_name_file}_{explainer_name}_{datasets_name}.json"
print(file_name)
# file_name = f"saliency_scores/bert_Integrated Gradient_HateXplain_saliency_Scores.json"
dataset,num_labels = load_dataset(datasets_name)
print(dataset)
# manager.compute_and_save_scores(dataset, file_name)
scores= manager.load_scores(file_path=file_name)

saliency_scores/bert_finetuned_Integrated Gradient_HateXplain.json


In [8]:
from evaluators.FAD import compute_all_fad
fad_score = compute_all_fad(dataset,model, tokenizer, scores)
print(f"FAD score: {fad_score}")

In [9]:
from evaluators.sensitivity import compute_all_sensitivity
sensitivity_score = compute_all_sensitivity(model, tokenizer,dataset, scores)
print(f"sensitivity score: {sensitivity_score}")

text ['<user> men can not be raped can not be abused that why they call it violence against women and children because men are always the perpetrators women come first even before children']
Before backward: perturbation.grad is None
After backward: perturbation.grad is tensor([[[ 1.0451e-04,  1.4380e-05,  1.5011e-04,  ...,  2.3417e-04,
           3.7909e-04,  1.7347e-04],
         [ 1.5607e-05,  1.4241e-04, -1.2863e-04,  ...,  2.4134e-04,
           2.1487e-04,  9.2079e-05],
         [ 2.1495e-04,  1.2225e-04, -1.9413e-05,  ...,  2.6517e-04,
           3.1639e-04,  1.2369e-04],
         ...,
         [-2.5920e-04, -3.2277e-04,  1.0502e-05,  ...,  2.7875e-04,
           7.7763e-05, -4.6208e-05],
         [-4.4934e-05, -2.7168e-04, -8.5538e-05,  ...,  5.2395e-04,
          -1.9053e-04,  1.5699e-04],
         [ 4.5053e-04,  2.2607e-04, -3.0830e-04,  ...,  8.6026e-05,
          -2.0363e-04, -4.0367e-04]]])
Step 0
Cumulative perturbation norm: 3.66607666015625
Before backward: perturbation

In [ ]:
from evaluators.recursiveROAR import compute_all_ROAR
roar_score = compute_all_ROAR(dataset, model, tokenizer, scores)
print(f"ROAR score: {roar_score}")

In [22]:
from evaluators.AUPRC import evaluate
auprc = evaluate(dataset,scores)
print(f"AUPRC: {auprc}")

Evaluating test split with AUPRC...
Average AUPRC Across Dataset: 0.5255
AUPRC: 0.5255247287200542


In [11]:
from evaluators.IOU import compute_all_IOU
IOU = compute_all_IOU(dataset,scores)
print(f"IOU: {IOU}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
IOU: {'IOU': np.float64(0.17907062163657347), 'F1': np.float64(0.25890798398702947)}


In [43]:
from evaluators.monotonicity import compute_all_monotonicity
monotonicity = compute_all_monotonicity(model, tokenizer, dataset,scores)
print(f"monotonicity: {monotonicity}")

0
1
2
3


KeyboardInterrupt: 

In [ ]:
for dataset_name in datasets_name:
    file_name = f"saliency_scores/{model_name_file}_{explainer_name}_{dataset_name}_saliency_scores.json"
    dataset,num_labels = load_dataset(dataset_name)
    manager.compute_and_save_scores(dataset=dataset,save_path=file_name)

dataset length 1924


100%|██████████| 1924/1924 [5:16:45<00:00,  9.88s/it]  


saliency data [{'text': ['<user> men can not be raped can not be abused that why they call it violence against women and children because men are always the perpetrators women come first even before children'], 'label': 'neutral', 'tokens': ['<s>', '▁<', 'user', '>', '▁men', '▁can', '▁not', '▁be', '▁rap', 'ed', '▁can', '▁not', '▁be', '▁abuse', 'd', '▁that', '▁why', '▁they', '▁call', '▁it', '▁violence', '▁against', '▁women', '▁and', '▁children', '▁because', '▁men', '▁are', '▁always', '▁the', '▁per', 'pet', 'ra', 'tors', '▁women', '▁come', '▁first', '▁even', '▁before', '▁children', '</s>'], 'saliency_scores': [0.04192134898034227, -0.042211258821179085, -0.044822470451521074, -0.004985074212299634, 0.0007309424933384708, 0.006394772656064059, 0.001193931841466984, 0.02179321470549705, 0.016479853446129462, -0.011813984357392192, -0.018819793628040628, -0.012267418082409812, -0.002021315902240513, 0.002393448718459284, -0.011779572335547635, -0.0008244928401964459, -0.05785309172191378, -

 57%|█████▋    | 113/199 [6:31:40<4:59:20, 208.84s/it]

In [26]:
manager.compute_and_save_scores(dataset=hatexplain,save_path="")

dataset length 1924


100%|██████████| 3/3 [00:31<00:00, 10.54s/it]

saliency data [{'text': ['<user> men can not be raped can not be abused that why they call it violence against women and children because men are always the perpetrators women come first even before children'], 'label': 'neutral', 'tokens': ['<s>', '▁<', 'user', '>', '▁men', '▁can', '▁not', '▁be', '▁rap', 'ed', '▁can', '▁not', '▁be', '▁abuse', 'd', '▁that', '▁why', '▁they', '▁call', '▁it', '▁violence', '▁against', '▁women', '▁and', '▁children', '▁because', '▁men', '▁are', '▁always', '▁the', '▁per', 'pet', 'ra', 'tors', '▁women', '▁come', '▁first', '▁even', '▁before', '▁children', '</s>'], 'saliency_scores': array([ 0.04192135, -0.04221126, -0.04482247, -0.00498507,  0.00073094,
        0.00639477,  0.00119393,  0.02179321,  0.01647985, -0.01181398,
       -0.01881979, -0.01226742, -0.00202132,  0.00239345, -0.01177957,
       -0.00082449, -0.05785309, -0.02766073,  0.00051816, -0.01647578,
       -0.08186529, -0.15260546, -0.00552901, -0.02208867, -0.04785234,
       -0.02084595, -0.03